In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pais;

CREATE TABLE Pais (
  nome VARCHAR(60),            -- eliminar ambiguidade!
  sigla VARCHAR(3) NOT NULL,
  regiao VARCHAR(30),
  classificacaoRenda VARCHAR(30),
  Gini INTEGER,
  PIBperCapita INTEGER,
  IDH INTEGER,
  populacao INTEGER,
  PRIMARY KEY(sigla)
) AS SELECT
    country,
    label,
    region,
    wb_class,
    gini,
    gdbPPPperCapita,
    hdi,
    populationTotal
FROM CSVREAD('../data/Filtered Data/countries.csv');

CREATE TABLE DST (
  regiao VARCHAR(30),
  ano INTEGER,
  quantidade DOUBLE,
  PRIMARY KEY(regiao, ano)
) AS SELECT
    region,
    year,
    nominal
FROM CSVREAD('../data/Filtered Data/infections.csv');

In [3]:
SELECT * FROM Pais;
SELECT * FROM DST;

In [4]:
SELECT D.ano, P.classificacaoRenda, CAST(SUM(D.quantidade) AS DOUBLE) / CAST(SUM(P.populacao) AS DOUBLE) taxaInfeccoes
FROM Pais P, DST D
WHERE P.sigla = D.regiao AND P.populacao IS NOT NULL 
GROUP BY D.ano, P.classificacaoRenda
ORDER BY TaxaInfeccoes DESC

In [5]:
SELECT * 
FROM PAIS
WHERE POPULACAO IS NOT NULL

In [6]:
SELECT TOP 3 * FROM PAIS;
SELECT TOP 3 * FROM DST;

In [16]:
CREATE VIEW tabelaInicial AS
SELECT D.ano, CAST(P.gini AS DOUBLE) / 1000.0 AS gini, 
       P.PIBperCapita, 
       CAST(D.quantidade AS DOUBLE) / CAST(P.populacao AS DOUBLE) taxaInfeccoes
FROM Pais P, DST D
WHERE P.sigla = D.regiao AND P.gini IS NOT NULL
      AND P.PIBperCapita IS NOT NULL 
      AND P.populacao IS NOT NULL
ORDER BY D.ano, gini, P.PIBperCapita

In [20]:
CREATE VIEW parNormInfeccoes AS
SELECT ano, 
       MIN(taxaInfeccoes) minInf, 
       MAX(taxaInfeccoes) maxInf
FROM tabelaInicial
GROUP BY ano
ORDER BY ano

In [23]:
CREATE VIEW parNormPibGini AS
SELECT MIN(pibpercapita) minPib, 
       MAX(pibpercapita) maxPib,
       MIN(1.0 - gini) minConvGini,
       MAX(1.0 - gini) maxConvGini
FROM tabelaInicial

In [25]:
SELECT T.ano, 
       (1.0 - T.gini - PNPG.minConvGini)/(PNPG.maxConvGini - PNPG.minConvGini) Corr_Gini,
       CAST(T.pibpercapita - PNPG.minpib AS DOUBLE) / CAST(PNPG.maxpib - PNPG.minpib AS DOUBLE) Norm_PIB,
       (T.taxaInfeccoes - PNI.minInf)/(PNI.maxInf-PNI.minInf) Norm_infeccoes
FROM tabelaInicial T, parNormInfeccoes PNI, parNormPibGini PNPG
WHERE T.ano = PNI.ano
ORDER BY ano, corr_gini, norm_pib